# Lab 02 - Parte 2 - PageRank 

In [2]:
import pandas as pd
import numpy as np
import decimal

In [3]:
data = pd.read_csv('soc-sign-bitcoinotc.csv', encoding='utf-8')
data = data.sort_values(by=['6', '2']) #ordenando o source e o target para contruir a matriz
data = data[data['4'] >= 8] #rating acima de 8
data.head()

,6,2,4,1289241911.72836
253,1,2,8,1.296629e+09
11988,1,4,10,1.343107e+09
4995,1,6,8,1.308242e+09
410,1,7,9,1.298267e+09
51,1,17,9,1.290969e+09


Feita a leitura dos dados do csv, ordenando os id's dos investidores e eliminando os que tinha notas abaixo de 8, vamos começar criando um dicionário onde as chaves serão os id's dos investidores, e a chave associada a cada um desses id's uma lista de todos os id's que a chave aponta.  

In [4]:
s = list(data['6']) #source
t = list(data['2']) #target
ids = list(set(s) | set(list(t))) #id's distintos

dic = {
    id_: [] for id_ in ids
}

for i in range(len(s)):
    source = s[i]
    target = t[i]
    dic[source].append(target)

Agora vamos ultilizar o dicionário criado para construir a matriz de transição H.

Abaixo a função que cria a matriz H a partir do dic.

In [10]:
def h_matrix(nodes):
    
    a = np.zeros(shape=(len(ids), len(ids)))
    
    for i in range(len(ids)):
        for j in range(len(ids)):
            source = ids[i]
            target = ids[j]
            if target in dic[source]:
                a[j][i] = float(1) / float(len(dic[source]))
                
    return np.matrix(a)

Agora vamos contruir todas as matrizes necessárias para o page rank

A matriz "a" que será a matriz H

A matriz "b" que é uma matriz de 1's multiplicada pelo escalar 1/quantidade de id's de investidores

A matriz "m" que é obtida da formula (1-d x a) + (d x b), onde d é o fator de teletransporte.

In [11]:
d = 0.15 #fator de teletransporte
a = h_matrix(ids) #matriz de transição
b = (float(1)/float(len(ids)))*np.ones(shape=(len(ids), len(ids))) #matriz de 1's
m = ((1-d)*a) + (d*b)

print ('m:','\n',m)

m: 
 [[0.00016393 0.09460838 0.00016393 ... 0.00016393 0.00016393 0.00016393]
 [0.07743666 0.00016393 0.00016393 ... 0.00016393 0.00016393 0.00016393]
 [0.00016393 0.00016393 0.00016393 ... 0.00016393 0.00016393 0.00016393]
 ...
 [0.00016393 0.00016393 0.00016393 ... 0.00016393 0.00016393 0.00016393]
 [0.00016393 0.00016393 0.00016393 ... 0.00016393 0.00016393 0.00016393]
 [0.00016393 0.00016393 0.00016393 ... 0.00016393 0.00016393 0.00016393]]


Agora o que resta é o vetor com os page rank de todos os investidores, que inicialmente serão todos iguais, 1/quantidade de investidores

In [13]:
#vetor com o page rank inicial dos investidores

v = (float(1)/len(ids))*np.ones(shape=(len(ids), 1))

A função para calcular o page rank dos investidores e contar o números de iterações que ocorreram

In [14]:
count = 0

def pageRank(v):
    global count
    if sum(abs(m*v-v)) > 0.001:
        count += 1
        return pageRank(m*v)
    else:
        count += 1
        return m*v

In [15]:
result = pageRank(v)
result = [cell.item(0,0) for cell in result]

Calculado o page rank de cada investidor vamos organizar e ordenar.

In [16]:
r = {}
for i in range(len(result)):
    r[ids[i]] = result[i]
    
r = sorted(r.items(), key=lambda x: x[1], reverse=True)

In [20]:
for i in r[0:5]:
    print('Id investidor:', i[0], ' Page Rank:', str(decimal.Decimal(i[1]))[:10])
    
print('Count:', count)

Id investidor: 1  Page Rank: 0.00013899
Id investidor: 202  Page Rank: 0.00012469
Id investidor: 144  Page Rank: 0.00011742
Id investidor: 3996  Page Rank: 0.00009278
Id investidor: 361  Page Rank: 0.00009154
Count: 35


# Perguntas

### Quantas iterações o PageRank precisou rodar até atingir convergência?

35

### Quais os 5 investidores mais importantes segundo o PageRank? Quais seus valores de PageRank?

Id investidor: 1         Page Rank: 0.00013899

Id investidor: 202       Page Rank: 0.00012469

Id investidor: 144       Page Rank: 0.00011742

Id investidor: 3996      Page Rank: 0.00009278

Id investidor: 361       Page Rank: 0.00009154

### Como você poderia usar o PageRank caso você fosse um investidor em bitcoins?

Quanto maior o page rank maior a quantidade de links para o investidor, então quanto maior page rank menos risco nas transações de com o investidor.

# Gráficos

### Id para Id

![title](id-id.png)

### Id para Page Rank

![title](id-pg.png)